# LangChain: Agents

## Outline:

* Using built in LangChain tools: Wikipedia
* Defining your own tools - Sum Calculator and Today date

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4o-mini"

## Built-in LangChain tools

In [3]:
%pip install -U wikipedia
%pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, create_tool_calling_agent, AgentExecutor
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [6]:
from langchain.tools.base import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field

class Sum(BaseModel):
    n: int = Field()
    k: int =Field()
    pass

def calculate_sum(n: int, k: int) -> int:
    return str(n+k)

calculate_sum=StructuredTool(
    name='calculate_sum',
    func=calculate_sum,
    description="Used to calculate sum",
    args_schema=Sum,
)

In [7]:
tools = load_tools(["wikipedia"], llm=llm)
tools.append(calculate_sum)
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/homebrew/anaconda3/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 StructuredTool(name='calculate_sum', description='Used to calculate sum', args_schema=<class '__main__.Sum'>, func=<function calculate_sum at 0x11e6ccf40>)]

In [8]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful agent that know how to solve questions using different tools.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [10]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
question = "What is the 25% of 300?"
result = agent_executor.invoke({
  "input": question
})
result.get("output")



> Entering new AgentExecutor chain...

Invoking: `calculate_sum` with `{'n': 300, 'k': 25}`


32525% of 300 is 75.

> Finished chain.


'25% of 300 is 75.'

## Wikipedia example

In [13]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent_executor.invoke({
  "input": question
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tom M. Mitchell'}`




/opt/homebrew/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footballer playing 

In [14]:
result.get("output")

'Tom M. Mitchell is the author of the textbook "Machine Learning."'

## Python Agent

In [15]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [16]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","ZZ"],
                 ["Jen","Ayai"]
                ]

In [17]:
response = agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
response.get("output")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


To sort the list of customers by last name and then by first name, I will use Python's built-in sorting capabilities. I will sort the list of lists using a key that sorts primarily by the last name and secondarily by the first name. 

Action: Python_REPL  
Action Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'ZZ'], ['Jen', 'Ayai']]; sorted_customers = sorted(customers, key=lambda x: (x[0], x[1])); print(sorted_customers)`  
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Jen', 'ZZ'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:I now know the final answer
Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Jen', 'ZZ'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


"[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Jen', 'ZZ'], ['Lang', 'Chain'], ['Trance', 'Former']]"

#### View detailed outputs of the chains

In [18]:
import langchain
langchain.debug=True
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'ZZ'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'ZZ'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execu

## Define your own tool

In [19]:
%pip install DateTime

Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.agents import tool
from datetime import date

In [21]:
class Time(BaseModel):
    text: str = Field()
    pass

def today_date(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

today_date_tool = StructuredTool(
    name='Time',
    func=today_date,
    description="Used to define today's date",
    args_schema=Time
  )

tools = load_tools(["wikipedia"], llm=llm)
tools.append(today_date_tool)
agent = create_tool_calling_agent(llm, tools, prompt)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [22]:
try:
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    result = agent_executor.invoke({
        "input": "What is the today date?"
    }) 
except: 
    print("exception on external access")

result.get("output")



> Entering new AgentExecutor chain...

Invoking: `Time` with `{'text': "today's date"}`


2024-07-27Today's date is July 27, 2024.

> Finished chain.


"Today's date is July 27, 2024."

Reminder: Download your notebook to you local computer to save your work.